## Open AI gym - MountainCar

### Open AI gym installeerimine

Piisab ka `pip3 install gym`, kuid sedasi ei saa kõiki erinevaid keskkondi alla laadida. Kõikide keskkondade installeerimiseks sobib:

```
git clone https://github.com/openai/gym.git
cd gym
pip3 install -e .
pip3 install -e '.[all]'
```

Rohkem infot: http://gym.openai.com/

In [ ]:
#!pip3 install gym tensorflow numpy keyboard matplotlib

In [1]:
import gym

env = gym.make('MountainCar-v0')

### MountainCar

Alustame lihtsamast - http://gym.openai.com/envs/MountainCar-v0/

> A car is on a one-dimensional track, positioned between two "mountains". The goal is to drive up the mountain on the right; however, the car's engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum.


- observation space (2) - position [-1.2, 0.6], velocity [-0.07, 0.07] (suunaline)
- eesmärk - position 0.5
- action space (3) - parem, vasak, *ära tee midagi*

### Teeme esmalt käsitsi juhitava klassi

Keyboard nõuab administraatori õigusi. `sudo` aitab: `sudo jupyter notebook --allow-root`

In [2]:
from keyboard import is_pressed

class HumanAgent:

    def __init__(self):
        self.actions = {
            'left': 0,
            'right': 2,
            'neutral': 1
        }
        
    def act(self, _):
        if is_pressed('left'):
            act = 'left'
        elif is_pressed('right'):
            act = 'right'
        else:
            act = 'neutral'

        return self.actions[act]


In [3]:
agent = HumanAgent()

for i in range(10):
    print('Episode', i, end='\r')
    observation = env.reset()
    done = False
    while not done:
        env.render()
        action = agent.act(observation)
        observation, reward, done, info = env.step(action)

env.close()

### Kuidas panna arvuti õppima?

Reeglina tähendab 'õppimine' masinõppe keeles mingi funktsiooni minimeerimist (vahel ka maksimeerimist). See tähendab, et esmalt tuleb probleem defineerida funktsioonina $y = f(X)$, kus y on soovitud tulemus, ning X sisendandmed. Masinõppes on arvuti ülesandeks leida $f(x)$, mille $y - \hat{y}$ on võimalikult väike. Teisisõnu püüab arvuti õppimise käigus minimeerida *õigete* tulemuste ja *ennustatud* tulemuste vahet.

Olenevalt ülesandest võib y juba eelnevalt olemas olla (juhendatud õpe; [*supervised learning*](https://en.wikipedia.org/wiki/Supervised_learning)), see võib puududa üleüldse (juhendamata õpe; [*unsupervised learning*](https://en.wikipedia.org/wiki/Unsupervised_learning)) või andmeid võidakse koguda käigult saades oma keskkonnalt tagasisidet (stiimulõpe; [*reinforcement learning*](https://en.wikipedia.org/wiki/Reinforcement_learning)). Open AI gym keskendub neist viimasele, pakkudes omalt poolt mitmeid keskkondi, mille abil andmeteadlased saavad õppida ja uusi masinõppe mudeleid leiutada.

Stiimulõppe arhitektuur koosneb lihtsustatult vaadeldavast keskkonnast (X-id), agendist (mudel), agendi tehtud otsustest ($\hat{y}$) ning saadud tagasisidest (y). Tagasisideks (*reward*) on langetatud otsuse väärtus ja seega on agendi ülesanne on õppida langetama otsuseid, mis maksimeerivad saadud tagasisidet.

![Reinforcement learning](Reinforcement_learning_diagram.svg)

Antud juhul on X-ideks `observation` (käru positsioon ja kiirus) ja soovitud tulemus `done` väärtus (positsioon == 0.5). Praktilises töös kulub valdav osa andmeteadlaste ajast X-ide ja y-te defineerimisele, sest mida selgem on seos sisendite ja väljundite vahel, seda lihtsam on arvutil õppida nende vaheline seos. Antud juhul on probleem  raskesti õpitav, sest eesmärk on diskreetne (*True / False*) ning enamuse ajast mitteinformatiivne. See, et suurema osa ajast on eesmärk saavutamata ei ütle midagi selle kohta, kui lähedal eesmärgi saavutamisele ollakse. Püüame seda muuta.

Ütleme, et eesmärgiks on hoopis maksimeerida liikumise kiirust, sest teame, et eesmärgi saavutamine on kiirusega korrelatsioonis ning kiirus annab meile pidevat tagasisidet, kui kaugel on eesmärgi saavutamine. Teiseks püüame panna arvutit õppima, kui suur on kiirus pärast ühe või teise valiku tegemist (*parem, vasak, ära tee midagi*), ehk mis on erinevate valikute väärtus tulevikus. Seda nimetatakse [q-learninguks](https://en.wikipedia.org/wiki/Q-learning) (q ehk *quality*). Täpsemalt näeb see valemi (*Bellmani võrrand*) kujul välja järgnev:


$$Q^{new}(s, a) = Q(s, a) + \alpha[R(s, a) + \gamma \max Q'(s', a') - Q(s, a)]$$

kus:
- Q - q-väärtus
- s - hetkeseisund (*state*)
- a - tehtud valik (*action*)
- $\alpha$ - õpisamm
- R - saadud kasu (*reward*)
- $\gamma$ - tuleviku realiseerumise tõenäosus
- $\max Q'(s', a') - Q(s, a)$ ennustatud maksimaalne tulevikus saadav lisandväärtus

Seame ülesse õppimisalgoritmi ja paneme q-valemi koodi.

In [4]:
import tensorflow
import numpy as np
import random
from collections import deque
tfk = tensorflow.keras

class AIAgent:
    """
    A simple neural network based AI agent.
    """
    def __init__(self, train, model_weights_dir=None):
        self.train = train
        self.memory = deque(maxlen=200)
        self.training_losses = deque(maxlen=10000)
        self.previous_state = None
        self.model_weights_dir = model_weights_dir
        self.model = self.create_model()
        
        self.alpha = 0.1
        self.gamma = 0.9
        
    def act(self, obs):
        """
        Outputs model actions given the state.
        """
        obs = self.reshape_single_obs(obs)
        pred = self.model.predict(obs)
        return np.argmax(pred)
    
    def create_model(self):
        """
        Defines the AI model.
        """
        model = tfk.Sequential([
            tfk.layers.Input(shape=(2)),
            tfk.layers.Dense(32, activation='relu'),
            tfk.layers.Dense(3, activation='linear')
        ], name='neural_net')
        
        model.compile(loss='mse', optimizer=tfk.optimizers.Adam(lr=1e-3))
        
        if self.model_weights_dir:
            try:
                model.load_weights(self.model_weights_dir)
                print('Pretrained weights initialized.')
            except:
                print('New weights initialized.')
            
        return model
        
    def save_memory(self, obs, reward, action, done):
        """
        Save episodes and Q values for training the model.
        """
        obs = self.reshape_single_obs(obs)
        
        # calculate new Q value if previous state exists
        if self.previous_state:
            current_q = self.previous_state['reward']
            predicted_q = np.max(self.model.predict(obs))
            
            new_q = current_q + self.alpha * (reward + self.gamma * predicted_q - current_q)
            targets = np.zeros((1,3))
            targets[0][action] = new_q
            
            self.memory.append((obs, targets))
        
        # train model on batch if memory is populated
        if self.train and len(self.memory) == self.memory.maxlen:
            self.train_model()
        
        # reset prev state if episode is completed
        if done:
            self.previous_state = None
        else:
            self.previous_state = {
                'obs': obs,
                'reward': reward,
                'action': action
            }
            
    def train_model(self, batch_size=32):
        """
        Trains the model on minibatch.
        """
        # sample random episides from memory
        batch = random.sample(self.memory, batch_size)
        X = []
        y = []
        for row in batch:
            X.append(row[0])
            y.append(row[1])
        X = np.vstack(X)
        y = np.vstack(y)
        loss = self.model.train_on_batch(X, y)
        self.training_losses.append(loss)
        if self.model_weights_dir:
            self.model.save(self.model_weights_dir)
            
    def reshape_single_obs(self,obs):
        """
        Reshapes a single observation to fit the model.
        """
        return obs.reshape(1,-1)

In [5]:
agent = AIAgent(train=True, model_weights_dir='model_checkpoints/dqn.h5')

for i in range(50):
    print('Episode', i + 1, 'Mean reward ...', end='\r')
    observation = env.reset()
    done = False
    rewards = []
    while not done:
        env.render()
        action = agent.act(observation)
        observation, reward, done, info = env.step(action)
        
        reward = abs(observation[1])**2
        agent.save_memory(observation, reward, action, done)
        rewards.append(reward)
    print('Episode', i + 1, 'Mean reward', round(np.mean(rewards),4))
        
env.close()

Pretrained weights initialized.


![MountainCar](mountain_car.gif)

In [6]:
#import matplotlib.pyplot as plt
#%matplotlib inline 
#%config InlineBackend.figure_format = 'png'
#plt.style.use('ggplot')
#
#plt.figure(figsize=(16,4))
#labels = ['total', 'left', 'neutral', 'right']
#for i in range(4):
#    plt.plot(np.vstack(agent.training_losses)[:,i], label=labels[i])
#plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4)
#plt.show()